In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import calendar
import datetime
import re
import os
import sys
from tqdm import tqdm

In [2]:
def get_cninfo_announcements(url, keywords, downloadDir, ts = 5):
    chrome_options = webdriver.ChromeOptions()
    prefs = {'profile.default_content_settings.popups': 0, #防止保存弹窗
        'download.default_directory':downloadDir,#设置默认下载路径
        'profile.default_content_setting_values.automatic_downloads':1 #允许多文件下载
        }
    chrome_options.add_experimental_option('prefs', prefs)
    
    browser = webdriver.Chrome(options=chrome_options)
    browser.get(url)
    
    searchBox = browser.find_element_by_xpath('//*[@id="main"]/div[3]/div/div[2]/div/div/div[2]/div[1]/div[1]/form/div[1]/div/div/div/input')
    searchBox.send_keys(keywords)
    searchBox.send_keys(Keys.ENTER)
    time.sleep(ts)
    
    data = browser.page_source
    pageNum = int(re.findall('<ul class="el-pager">.*>(.*?)</li></ul><button type',data)[0])
    
    datas = []
    datas.append(data)
    for index in range(1,pageNum):
        browser.find_element_by_xpath('//*[@id="main"]/div[3]/div/div[2]/div/div/div[2]/div[1]/div[4]/div/div/button[2]').click()
        time.sleep(ts)
        data = browser.page_source
        datas.append(data)
    allData = ''.join(datas)
    
    p_title = '<div class="cell"><a target="_blank" href=".*?" data-id=".*?" data-orgid=".*?" data-seccode=".*?" class="ahover"><span title="(.*?)" class="r-title">.*?</div>'
    p_href = '<div class="cell"><a target="_blank" href="(.*?)" data-id=".*?" data-orgid=".*?" data-seccode=".*?" class="ahover"><span title=".*?" class="r-title">.*?</div>'

    p_bulletId = '<div class="cell"><a target="_blank" href=".*?" data-id="(.*?)" data-orgid=".*?" data-seccode=".*?" class="ahover"><span title=".*?" class="r-title">.*?</div>'
    p_date = '<div class="cell"><a target="_blank" href=".*?announcementTime=(.*?)" data-id=".*?" data-orgid=".*?" data-seccode=".*?" class="ahover"><span title=".*?" class="r-title">.*?</div>'
 
    title = re.findall(p_title,allData)
    href = re.findall(p_href,allData)
    bulletId = re.findall(p_bulletId,allData) 
    date = re.findall(p_date,allData)
    
    for index in tqdm(range(len(title))):
#        print(title[index],' - ',date[index])
#        print(downloadUrl)  
        downloadUrl = 'http://www.cninfo.com.cn/new/announcement/download?bulletinId='+bulletId[index]+'&announceTime='+date[index]
        browser.get(downloadUrl)
        time.sleep(1)
        sys.stdout.flush()
    
    browser.close()
    

In [3]:
url = 'http://www.cninfo.com.cn/new/disclosure/stock?stockCode=002714&orgId=9900022995#latestAnnouncement'
downloadDir = os.getcwd()+"\\announcements\\muyuan\\monthly"
get_cninfo_announcements(url, '销售简报', downloadDir)

100%|██████████| 100/100 [01:57<00:00,  1.17s/it]
